Load the houseprices data from Thinkful's database.


In [7]:
%reload_ext nb_black


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt

from sqlalchemy import create_engine

import warnings

import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse

warnings.filterwarnings("ignore")
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "houseprices"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
house_df = pd.read_sql_query("select * from houseprices", con=engine)

engine.dispose()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Reimplement your model from the previous checkpoint.


In [2]:
house_df = pd.concat([house_df, pd.get_dummies(house_df.housestyle)], axis=1)
dummy = list(pd.get_dummies(house_df.housestyle).columns)
X = house_df[["overallqual", "grlivarea", "garagecars"] + dummy]

X = X[
    [
        "overallqual",
        "grlivarea",
        "garagecars",
        "1.5Fin",
        "1Story",
        "2.5Fin",
        "2.5Unf",
        "2Story",
        "SFoyer",
    ]
]
Y = house_df["saleprice"]

<IPython.core.display.Javascript object>

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=27
)
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     527.3
Date:                Wed, 02 Sep 2020   Prob (F-statistic):               0.00
Time:                        12:28:58   Log-Likelihood:                -17484.
No. Observations:                1460   AIC:                         3.499e+04
Df Residuals:                    1450   BIC:                         3.504e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -1.13e+05   6277.246    -18.005      0.000   -1.25e+05   -1.01e+05
overallqual  2.593e+04   1031.701     25.137      0.000    2.39e+04     2.8e+04
grlivarea      70.7127      2.939     24.060      0.000      64.948      76.478
garagecars   1.646e+04   1773.772      9.277      0.000     1.3e+04    1.99e+04
1.5Fin      -1.579e+04   5481.682     -2.881      0.004   -2.65e+04   -5039.596
1Story       1.325e+04   4571.239      2.898      0.004    4280.913    2.22e+04
2.5Fin      -5.838e+04    1.5e+04     -3.897      0.000   -8.78e+04    -2.9e+04
2.5Unf       -5.82e+04   1.25e+04     -4.642      0.000   -8.28e+04   -3.36e+04
2Story      -1.655e+04   4911.176     -3.370      0.001   -2.62e+04   -6915.939
SFoyer       1.583e+04   7719.674      2.051      0.040     690.285     3.1e+04
==============================================================================
Omnibus:                      358.053   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19899.291
Skew:                           0.068   Prob(JB):                         0.00
Kurtosis:                      21.086   Cond. No.                     2.54e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.54e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<IPython.core.display.Javascript object>

Try OLS, Lasso, Ridge, and ElasticNet regression using the same model specification. This time, you need to do k-fold cross-validation to choose the best hyperparameter values for your models. Scikit-learn has RidgeCV, LassoCV, and ElasticNetCV that you can utilize to do this. Which model is the best? Why?

In [11]:
alphas = [np.power(10.0, p) for p in np.arange(-10, 40, 1)]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
lasso_cv = LassoCV(alphas=alphas, cv=5)

lasso_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lasso_cv.predict(X_train)
y_preds_test = lasso_cv.predict(X_test)

print("Best alpha value is: {}".format(lasso_cv.alpha_))
print(
    "R-squared of the model in training set is: {}".format(
        lasso_cv.score(X_train, y_train)
    )
)
print("-----Test set statistics-----")
print(
    "R-squared of the model in test set is: {}".format(lasso_cv.score(X_test, y_test))
)
print(
    "Mean absolute error of the prediction is: {}".format(
        mean_absolute_error(y_test, y_preds_test)
    )
)
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print(
    "Root mean squared error of the prediction is: {}".format(
        rmse(y_test, y_preds_test)
    )
)
print(
    "Mean absolute percentage error of the prediction is: {}".format(
        np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100
    )
)

Best alpha value is: 100.0
R-squared of the model in training set is: 0.7594141275118893
-----Test set statistics-----
R-squared of the model in test set is: 0.7752026930678922
Mean absolute error of the prediction is: 26320.739262559047
Mean squared error of the prediction is: 1507277117.1435916
Root mean squared error of the prediction is: 38823.666971881874
Mean absolute percentage error of the prediction is: 16.121026586582623


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
ridge_cv = RidgeCV(alphas=alphas, cv=5)

ridge_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print("Best alpha value is: {}".format(ridge_cv.alpha_))
print(
    "R-squared of the model in training set is: {}".format(
        ridge_cv.score(X_train, y_train)
    )
)
print("-----Test set statistics-----")
print(
    "R-squared of the model in test set is: {}".format(ridge_cv.score(X_test, y_test))
)
print(
    "Mean absolute error of the prediction is: {}".format(
        mean_absolute_error(y_test, y_preds_test)
    )
)
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print(
    "Root mean squared error of the prediction is: {}".format(
        rmse(y_test, y_preds_test)
    )
)
print(
    "Mean absolute percentage error of the prediction is: {}".format(
        np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100
    )
)

Best alpha value is: 1.0
R-squared of the model in training set is: 0.7602294369359575
-----Test set statistics-----
R-squared of the model in test set is: 0.7801889958433517
Mean absolute error of the prediction is: 26076.79183374127
Mean squared error of the prediction is: 1473843709.176345
Root mean squared error of the prediction is: 38390.67216364341
Mean absolute percentage error of the prediction is: 15.960544227755763


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
elasticnet_cv = ElasticNetCV(alphas=alphas, cv=5)
elasticnet_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticnet_cv.predict(X_train)
y_preds_test = elasticnet_cv.predict(X_test)

print("Best alpha value is: {}".format(elasticnet_cv.alpha_))
print(
    "R-squared of the model in training set is: {}".format(
        elasticnet_cv.score(X_train, y_train)
    )
)
print("-----Test set statistics-----")
print(
    "R-squared of the model in test set is: {}".format(
        elasticnet_cv.score(X_test, y_test)
    )
)
print(
    "Mean absolute error of the prediction is: {}".format(
        mean_absolute_error(y_test, y_preds_test)
    )
)
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print(
    "Root mean squared error of the prediction is: {}".format(
        rmse(y_test, y_preds_test)
    )
)
print(
    "Mean absolute percentage error of the prediction is: {}".format(
        np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100
    )
)

Best alpha value is: 0.01
R-squared of the model in training set is: 0.759497725873024
-----Test set statistics-----
R-squared of the model in test set is: 0.775958984465708
Mean absolute error of the prediction is: 26276.815500145152
Mean squared error of the prediction is: 1502206145.7277088
Root mean squared error of the prediction is: 38758.304216357414
Mean absolute percentage error of the prediction is: 16.100900400667502


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

It appears that the best model is the ridge model.